## Pyswip-notebook usage examples

Spawn the interpreter instance and add some initial facts. SWI-Prolog will be started automatically in the background.

The facts can be loaded directly from a string as well.



In [1]:
from pyswip_notebook import IsolatedProlog

prolog = IsolatedProlog()


facts = """
parent(cronus,hestia).
parent(cronus,pluto).
parent(cronus,poseidon).
parent(cronus,zeus).
parent(cronus,hera).
parent(cronus,demeter).
parent(rhea, hestia).
parent(rhea, pluto).
parent(rhea, poseidon).
parent(rhea, zeus).
parent(rhea, hera).
parent(rhea, demeter).

parent(zeus, athena).

parent(zeus, ares).
parent(zeus, hebe).
parent(zeus, hephaestus).
parent(hera, ares).
parent(hera, hebe).
parent(hera, hephaestus).

parent(zeus, persephone).
parent(demeter, persephone).


male(cronus).
male(pluto).
male(poseidon).
male(zeus).
male(ares).
male(hephaestus).
female(rhea).
female(hestia).
female(hera).
female(demeter).
female(athena).
female(hebe).
female(persephone).
"""

prolog.consult(facts, file=False) 


Print all facts in the knowledge base.

In [4]:
# 
for l in prolog.query("listing"):
    print(l)


female(rhea).
female(hestia).
female(hera).
female(demeter).
female(athena).
female(hebe).
female(persephone).

parent(cronus, hestia).
parent(cronus, pluto).
parent(cronus, poseidon).
parent(cronus, zeus).
parent(cronus, hera).
parent(cronus, demeter).
parent(rhea, hestia).
parent(rhea, pluto).
parent(rhea, poseidon).
parent(rhea, zeus).
parent(rhea, hera).
parent(rhea, demeter).
parent(zeus, athena).
parent(zeus, ares).
parent(zeus, hebe).
parent(zeus, hephaestus).
parent(hera, ares).
parent(hera, hebe).
parent(hera, hephaestus).
parent(zeus, persephone).
parent(demeter, persephone).

male(cronus).
male(pluto).
male(poseidon).
male(zeus).
male(ares).
male(hephaestus).
{}


Next we add a few more complext rules.

In [6]:
prolog.assertz("isFather(X, Y) :- male(X), parent(X, Y)")

prolog.assertz("isMother(X, Y) :- female(X), parent(X, Y)")

prolog.assertz("isDaughter(X, Y) :- female(X), parent(Y, X)")

prolog.assertz("isSon(X, Y) :- male(X), parent(Y, X)")

prolog.assertz("isAncestor(X, Y) :- parent(X, Y)")

prolog.assertz("isAncestor(X, Y) :- parent(X, T), parent(T, Y)")

We can query the knowledge base.

In [7]:
# 
res = list(prolog.query("isAncestor(rhea, persephone)"))
print("false" if len(res) == 0 else "true")

true


In [8]:
# a more complext query with a variable
for res in prolog.query("isFather(zeus, Y)"):
    print(res)

{'Y': 'athena'}
{'Y': 'ares'}
{'Y': 'hebe'}
{'Y': 'hephaestus'}
{'Y': 'persephone'}


Multiple instances of `IsolatedProlog` do not interfere with each other.

In [9]:
# 

another_prolog = IsolatedProlog()

another_prolog.assertz("is_empty(another_prolog)")

for l in another_prolog.query("listing"):
    print(l)

{}

:- dynamic is_empty/1.

is_empty(another_prolog).


Foreign functions from `pyswip` work as well

In [12]:
from pyswip import registerForeign

def hello(t):
    print("Hello,", t)
hello.arity = 1

registerForeign(hello)

prolog = IsolatedProlog()
prolog.assertz("father(michael,john)")
prolog.assertz("father(michael,gina)")
print(list(prolog.query("father(michael,X), hello(X)")))

Hello, john
Hello, gina
[{'X': 'john'}, {'X': 'gina'}]
